# Transfer Learning

Closely modeled after fast.ai's [NLP course](https://github.com/fastai/course-nlp).

### Settings & Imports

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai import *
from fastai.text import *
import os.path

### Data Preprocessing

In [8]:
path = "./Replications/Davidson2017/data"

In [29]:
bs = 32  # Batch size

# If files have been saved, load them.
if os.path.exists(path + "/data_lm_export.pkl"):
    
    print("Loading saved data files...\n")
    
    data_lm = load_data(path, 'data_lm_export.pkl')
    data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)
    
else:
    
    print("Files not found. Creating data files...\n")
    
    # Language model data
    data_lm = TextLMDataBunch.from_csv(path, 'labeled_data.csv', text_cols='tweet')
    # Classifier model data
    data_clas = TextClasDataBunch.from_csv(path, 'labeled_data.csv', text_cols='tweet', label_cols='class', 
                                           vocab = data_lm.train_ds.vocab, bs=bs)
    
    # Save files
    data_lm.save('data_lm_export.pkl')
    data_clas.save('data_clas_export.pkl')

Loading saved data files...



In [30]:
data_lm.show_batch()


idx,text
0,"security , please ignore any foreign threats and focus your sole attention on our police force # xxup xxunk # xxup xxunk xxbos xxmaj this bitch @lvl_7_eevee buys xxunk water . xxbos xxunk you a bitch tho i know for sure i 'm not . xxmaj you 'd get your ass beat by any of my bitches that 's fact 😂 😂 xxbos xxmaj that 's yo bitch xxrep 5"
1,then be xxunk when she taking care of it by herself ? xxbos i coulda xxunk i was the last bitch on tunchi ig 💁 ✌ ️ 😘 xxbos @nasshrodriguez @linndaroose always down patty never been a pussy 😼 xxbos “ xxunk _ : xxmaj that pussy so wet she got ta dry xxunk telling my business xxbos i 'm hooked on monkey xxunk . xxbos a hoe gon na
2,rick 's positions are xxunk by xxunk … xxbos xxmaj these hoes is lost xxup rt xxunk : @1bcarter hahaha how much you wanna bet she do n't get the job lmao xxbos xxmaj what ever happened to the xxmaj teen xxmaj titans of my day and why must they mock me with this xxmaj teen xxmaj titans xxmaj go ? xxbos “ @liidurk _ _ : xxmaj females get
3,"t.co / xxunk xxbos xxmaj xxunk xxmaj banks still getting paid for these xxmaj america 's next top model re runs , this bitch so bossy 👠 xxunk xxbos xxmaj leave it up to me it would 've been in the trash xxrep 4 . xxmaj all of it . xxbos xxmaj wallah i 'll drop kick this bitch xxbos xxmaj crystal got all the hoes . xxbos @coolstoryisaac hahaha"
4,than xxunk you want but aye i 'm gon na have fun bitch so xxup suck a xxup xxunk ! ! xxbos “ xxunk xxmaj and happy to be hoes too . xxmaj xxunk than being in denial xxbos @denytheprophecy that 's a dragon you retard xxbos # xxunk has a highschool graduation rate near 50 % and that 's why they have so many uninformed # teabaggers # morningjoe


In [31]:
data_clas.show_batch()

text,target
xxbos xxmaj was finna slit my eyebrows up in the shop but na xxrep 6 h . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n,2
xxbos xxup rt @heissarcastic : xxmaj if my jokes offend you : \n \n 1 ) i 'm sorry \n 2 ) xxmaj it wo n't happen again \n 3 ) 1 & & 2 are lies \n 4 ) xxmaj you 're a pussy \n 5 ) xxmaj your xxmaj mom hates you …,1
"xxbos xxup rt @fewjr : i do n't trust nobody . 😔 👌 \n \n xxmaj these bitches fake . 👉 💃 \n \n xxmaj these niggas faker . 👉 🚶 \n \n xxmaj so it 's not "" fuck y' all "" , i 'd rather just not fuck wi …",1
xxbos xxup rt xxunk : xxmaj all a nigga need is money . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n xxmaj fuck a bitch,1
"xxbos xxmaj lmaoo “ @leanandcuisine : xxmaj bad bitch : xxmaj can i hit that ? \n \n "" xxmaj you matching or xxrep 5 . ? "" \n \n xxmaj bad bitch : * bats eyelashes * \n \n "" xxrep 9 . are you matching or ? "" ”",1
